# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# Imports and setup
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
if api_key and api_key.startswith("sk-") and len(api_key) > 10:
    print("API key found")
else:
    print("No OpenAI API key found / set OPENAI_API_KEY in .env for cloud models")

# OpenAI (cloud) and Ollama (local) clients
openai_client = OpenAI(api_key=api_key) if api_key else None
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama_client = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

# Models: (display_name, api_model_id, use_openai)
MODELS = [
    ("GPT-4o mini (OpenAI)", "gpt-4o-mini", True),
    ("Llama 3.2 (Ollama)", "llama3.2", False),
]
MODEL_CHOICES = [m[0] for m in MODELS]

In [47]:
# System prompt: technical Q&A expertise
SYSTEM_PROMPT = """You are a helpful assistant that explains technical concepts clearly, including for non-technical audiences.
You can use simple analogies, step-by-step reasoning, and examples. When asked for the current date or time, use the get_current_datetime tool.
Respond in markdown when useful (code snippets, lists). Be concise but complete."""

In [48]:
# Tool: get_current_datetime (bonus – demonstrates tool use)
def get_current_datetime(timezone_hint: str = "") -> str:
    """Return current date and time. Optional timezone hint (e.g. 'UTC', 'EST') for context."""
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S") + (f" (hint: {timezone_hint})" if timezone_hint else "")

tools_schema = [
    {
        "type": "function",
        "function": {
            "name": "get_current_datetime",
            "description": "Get the current date and time. Use when the user asks for today's date, current time, or similar.",
            "parameters": {
                "type": "object",
                "properties": {
                    "timezone_hint": {"type": "string", "description": "Optional timezone hint, e.g. UTC, EST"},
                },
                "additionalProperties": False,
            },
        },
    }
]

In [49]:
def _get_client_and_model(choice: str):
    """Resolve dropdown choice to (client, model_id, use_tools)."""
    for display_name, model_id, use_openai in MODELS:
        if choice == display_name:
            client = openai_client if use_openai else ollama_client
            return client, model_id, use_openai
    return openai_client, MODELS[0][1], True

def _handle_tool_calls(message, messages):
    """Append tool results to messages. Returns updated messages."""
    for tool_call in message.tool_calls or []:
        if tool_call.function.name == "get_current_datetime":
            args = json.loads(tool_call.function.arguments or "{}")
            result = get_current_datetime(args.get("timezone_hint", ""))
            messages.append({
                "role": "tool",
                "content": result,
                "tool_call_id": tool_call.id,
            })
    return messages

In [52]:
def chat_stream(message, history, model_choice: str):
    """
    Chat with streaming. Yields updated message history for the chatbot.
    Supports model switching and (for OpenAI) tool use.
    """
    client, model_id, use_openai = _get_client_and_model(model_choice)
    
    # History already includes the latest user message from the UI
    history_list = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history_list
    # Last item is the user message we're responding to
    new_user = history_list[-1] if history_list else {"role": "user", "content": message}
    new_assistant = {"role": "assistant", "content": ""}

    if use_openai and openai_client:
        # OpenAI: use non-streaming when tools are involved, then stream final text
        kwargs = {"model": model_id, "messages": messages, "tools": tools_schema}
        response = client.chat.completions.create(**kwargs)
        while response.choices[0].finish_reason == "tool_calls":
            msg = response.choices[0].message
            messages.append({"role": "assistant", "content": msg.content or "", "tool_calls": [
                {"id": tc.id, "type": "function", "function": {"name": tc.function.name, "arguments": tc.function.arguments}}
                for tc in msg.tool_calls
            ]})
            for tc in msg.tool_calls:
                if tc.function.name == "get_current_datetime":
                    args = json.loads(tc.function.arguments or "{}")
                    messages.append({"role": "tool", "content": get_current_datetime(args.get("timezone_hint", "")), "tool_call_id": tc.id})
            response = client.chat.completions.create(model=model_id, messages=messages, tools=tools_schema)
        full_content = response.choices[0].message.content or ""
        # Stream the final text into the UI (yield progressively)
        for i in range(1, len(full_content) + 1):
            new_assistant["content"] = full_content[:i]
            yield history + [dict(new_assistant)]
        return
    else:
        # Ollama: simple streaming, no tools
        stream = client.chat.completions.create(model=model_id, messages=messages, stream=True)
        accumulated = ""
        for chunk in stream:
            part = (chunk.choices[0].delta.content or "") if chunk.choices else ""
            accumulated += part
            new_assistant["content"] = accumulated
            yield history + [dict(new_assistant)]

In [ ]:
# Gradio UI: chatbot, model dropdown, streaming
def add_user_to_chat(message, history):
    """On submit: clear input and append user message to history."""
    return "", history + [{"role": "user", "content": message}]

with gr.Blocks(title="Funky Tech Duo", theme=gr.themes.Soft()) as demo:
    gr.Markdown("## Funky Tech Duo\nAsk us anything: *Llama the wise shut-in & GPT the impatient know-it-all (even datetime).*")
    chatbot = gr.Chatbot(type="messages", height=400, label="Chat")
    msg_in = gr.Textbox(placeholder="Ask a question...", label="Message", show_label=False)
    model_dropdown = gr.Dropdown(choices=MODEL_CHOICES, value=MODEL_CHOICES[0], label="Model")
    clear_btn = gr.Button("Clear")

    msg_in.submit(
        add_user_to_chat,
        inputs=[msg_in, chatbot],
        outputs=[msg_in, chatbot],
    ).then(
        chat_stream,
        inputs=[msg_in, chatbot, model_dropdown],
        outputs=chatbot,
    )
    clear_btn.click(lambda: [], outputs=[chatbot])

demo.launch(inbrowser=True)